In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import
import pandas as pd 
import numpy as np
import os
import librosa

from pydub import AudioSegment
from pydub.playback import play
from pydub.utils import make_chunks

import matplotlib.pyplot as plt
import librosa.display

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, train_test_split, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


# Function Explanation
(name of func.): explain

- convert_data(file_path, before_format, after_format): convert original file`s format
- segment_data(main_path, file_path, save_path, sec): The data is  segmented by sec



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mainPath = "/content/drive/Shareddrives/AI-Drone/Dataset/Outdoor/AudioV2/originalSeparate"
pathList = ["(-5, 5)", "(-5, 10)", "(-5, 15)", "(0, 5)", "(0, 10)", "(0, 15)", "(5, 5)", "(5, 10)", "(5, 15)"]
pathValue = "mfcc(value)"


df = {}
countUAV = 0
countNone = 0

for idx, f in enumerate(os.listdir(os.path.join(mainPath, pathList[0], pathValue))):
  
  mfccs = pd.read_csv(os.path.join(mainPath, pathList[0], pathValue, f), index_col=0)
  
  # 1D
  pca = PCA(n_components=1)
  scaler = StandardScaler()


  # Normalization (type: ndarray)
  std_mfccs = scaler.fit_transform(mfccs)
  
  # Transepose (mfcc x time) to (time x mfcc)
  trans_mfccs = std_mfccs.T
  print(trans_mfccs.shape)
  # PCA
  pca = pca.fit_transform(trans_mfccs) 
  
  for p in pca:
    if idx not in df:
      df[idx] = []
    df[idx].append(p[0])

  # Label
  if "Small" in f or "Big" in f:
    df[idx].append(1)
    countUAV +=1
  elif "No" in f:
    df[idx].append(0)
    countNone += 1

  print(f)

print(countUAV, countNone)

df = pd.DataFrame.from_dict(df, orient='index')
df.to_csv(os.path.join(mainPath, pathList[0], "MFCC.csv"))
  

In [ ]:
# Set the parameters by cross-validation
tuned_parameters =  [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.01, 0.1, 10, 25, 50, 100, 1000]},
                    {'kernel': ['sigmoid'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.01, 0.1, 10, 25, 50, 100, 1000] },
                    {'kernel': ['linear'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}]              


clf = GridSearchCV(SVC(), tuned_parameters, cv=5, scoring="accuracy")

In [ ]:
mainPath = "/content/drive/Shareddrives/AI-Drone/Dataset"
pathList = ["0516 Indoor", "0517 Indoor", "0518 Indoor", "0519 Indoor"]
pathValue = "mfcc(value)"

new_df1 = pd.read_csv(os.path.join(mainPath, pathList[0], "MFCC.csv"), index_col=0)
new_df2 = pd.read_csv(os.path.join(mainPath, pathList[1], "MFCC.csv"), index_col=0)
new_df3 = pd.read_csv(os.path.join(mainPath, pathList[2], "MFCC.csv"), index_col=0)
new_df4 = pd.read_csv(os.path.join(mainPath, pathList[3], "MFCC.csv"), index_col=0)

temp = pd.concat([new_df1, new_df2], ignore_index=True)
temp = pd.concat([temp, new_df3], ignore_index=True)
df = pd.concat([temp, new_df4], ignore_index=True)
print(len(df), df.columns)

1874 Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20'],
      dtype='object')


In [ ]:
X = df.drop(columns=['20'])
y = df['20']


In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
clf.fit(X, y)
print(clf.best_params_)
print(clf.best_score_) 

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
0.8660720142602495


In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

def rf(X, y, cv):
  # Prepare for grid_search
  param_grid = { 
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [i for i in range(2, 16)],
    'criterion' :['gini', 'entropy']
  }
  # Setting
  rfc=RandomForestClassifier() # random_state=42
  clf = GridSearchCV(rfc, param_grid=param_grid, cv=cv, scoring="accuracy")


  # Training
  clf.fit(X, y)

  return clf

In [ ]:
# KNN
def knn(X, y, cv):
  # Prepare for grid_search
  k_range = list(range(1, 31))
  param_grid = dict(n_neighbors=k_range)

  # Setting
  knc = KNeighborsClassifier()
  clf = GridSearchCV(knc, param_grid, cv=cv, scoring="accuracy") # , scoring="accuracy"

  # Training
  clf.fit(X, y)

  return clf

In [ ]:
# Exec

clf_knn = knn(X, y, 5)
print(clf_knn.best_params_)
print(clf_knn.best_score_) 

clf_rf = rf(X, y, 5)
print(clf_rf.best_params_)
print(clf_rf.best_score_) 
